In [1]:
import pandas as pd
import tensorflow as tf
from tensorflow.keras import Sequential
from tensorflow.keras.layers import Dense

import numpy as np
import json

from sklearn.model_selection import train_test_split
from sklearn.utils import shuffle
from google.cloud import bigquery
from witwidget.notebook.visualization import WitWidget, WitConfigBuilder

In [3]:
query="""
SELECT
  weight_pounds,
  is_male,
  mother_age,
  plurality,
  gestation_weeks
FROM
  publicdata.samples.natality
WHERE year > 2000
LIMIT 10000
"""
df = bigquery.Client().query(query).to_dataframe()
df.head()

,weight_pounds,is_male,mother_age,plurality,gestation_weeks
0,6.686620,True,18,1,43.0
1,9.360828,True,32,1,41.0
2,8.437091,False,30,1,39.0
3,6.124442,False,24,1,40.0
4,7.125340,False,26,1,41.0


In [4]:
df.describe()

,weight_pounds,mother_age,plurality,gestation_weeks
count,9989.000000,10000.000000,10000.000000,9890.000000
mean,7.297602,27.298900,1.034400,38.699798
std,1.291685,6.165838,0.192926,2.539957
min,0.612885,12.000000,1.000000,17.000000
25%,6.624891,22.000000,1.000000,38.000000
50%,7.374463,27.000000,1.000000,39.000000
75%,8.124034,32.000000,1.000000,40.000000
max,12.257702,50.000000,3.000000,47.000000


In [6]:
df = df.dropna()
df = shuffle(df, random_state=2)

In [7]:
labels = df['weight_pounds']
data = df.drop(columns=['weight_pounds'])
data['is_male'] = data['is_male'].astype(int)

In [8]:
data.head()

,is_male,mother_age,plurality,gestation_weeks
8103,1,27,1,40.0
9816,0,36,2,34.0
3310,1,36,1,36.0
4843,1,25,1,39.0
5719,0,39,1,38.0


In [9]:
x,y = data,labels
x_train,x_test,y_train,y_test = train_test_split(x,y)

In [10]:
model = Sequential([
    Dense(64, activation='relu', input_shape=(len(x_train.iloc[0]),)),
    Dense(32, activation='relu'),
    Dense(1)]
)

In [11]:
model.compile(optimizer=tf.keras.optimizers.RMSprop(),
              loss=tf.keras.losses.MeanSquaredError(),
              metrics=['mae', 'mse'])

In [12]:
model.fit(x_train, y_train, epochs=10, validation_split=0.1)

Train on 6670 samples, validate on 742 samples
Epoch 1/10
6670/6670 [==============================] - 2s 263us/sample - loss: 5.0748 - mae: 1.3098 - mse: 5.0748 - val_loss: 1.2615 - val_mae: 0.8771 - val_mse: 1.2615
Epoch 2/10
6670/6670 [==============================] - 1s 138us/sample - loss: 1.3052 - mae: 0.8952 - mse: 1.3052 - val_loss: 1.1782 - val_mae: 0.8489 - val_mse: 1.1782
Epoch 3/10
6670/6670 [==============================] - 1s 159us/sample - loss: 1.2989 - mae: 0.8904 - mse: 1.2989 - val_loss: 1.0964 - val_mae: 0.8293 - val_mse: 1.0964
Epoch 4/10
6670/6670 [==============================] - 1s 166us/sample - loss: 1.2730 - mae: 0.8833 - mse: 1.2730 - val_loss: 1.1214 - val_mae: 0.8277 - val_mse: 1.1214
Epoch 5/10
6670/6670 [==============================] - 1s 174us/sample - loss: 1.2692 - mae: 0.8819 - mse: 1.2692 - val_loss: 1.1704 - val_mae: 0.8631 - val_mse: 1.1704
Epoch 6/10
6670/6670 [==============================] - 1s 155us/sample - loss: 1.2363 - mae: 0.8678 - 

In [13]:
num_examples = 10
predictions = model.predict(x_test[:num_examples])

In [14]:
for i in range(num_examples):
    print('Predicted val: ', predictions[i][0])
    print('Actual val: ',y_test.iloc[i])
    print()

Predicted val:  7.5644593
Actual val:  7.1870697412

Predicted val:  6.7632785
Actual val:  4.7509617461

Predicted val:  7.382397
Actual val:  6.686620406459999

Predicted val:  7.7864203
Actual val:  7.5618555866

Predicted val:  7.6117506
Actual val:  8.313631900019999

Predicted val:  7.31854
Actual val:  7.06361087448

Predicted val:  7.48036
Actual val:  7.7051560569

Predicted val:  6.7799783
Actual val:  7.7492485093

Predicted val:  7.717244
Actual val:  7.12534030784

Predicted val:  6.9452424
Actual val:  7.12534030784



In [15]:
wit_data = pd.concat([x_test, y_test], axis=1)

In [16]:
def custom_predict(examples_to_infer):
    preds = model.predict(examples_to_infer)
    return preds

In [17]:
config_builder = (WitConfigBuilder(wit_data[:500].values.tolist(), data.columns.tolist() + ['weight_pounds'])
  .set_custom_predict_fn(custom_predict)
  .set_target_feature('weight_pounds')
  .set_model_type('regression'))
WitWidget(config_builder, height=800)

WitWidget(config={'model_type': 'regression', 'label_vocab': [], 'feature_names': ['is_male', 'mother_age', 'p…